# Brochure Generator from Website

In [ ]:
#all imports
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
import google.generativeai as genai

## Step 1: Get data and links

In [ ]:
load_dotenv(override=True)
api_key = os.getenv('GEMINI_API_KEY')

# connect to gemini
genai.configure(api_key = api_key)

In [ ]:

# Some websites need you to use proper headers when fetching them:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links.
    """
    url: str
    title: str
    body: str
    text: str
    links: List[str]

    def __init__(self, url: str):
        self.url = url
        self.title = ""
        self.body = ""
        self.text = ""
        self.links = []
        self.scrape()

    def scrape(self):
        """Scrapes the content of the website at the provided URL."""
        try:
            response = requests.get(self.url, headers=headers)
            if response.status_code != 200:
                print(f"Failed to retrieve the URL. Status code: {response.status_code}")
                return
            
            soup = BeautifulSoup(response.content, 'html.parser')
            self.body = response.content
            self.title = soup.title.string if soup.title else "No title found!"

            # Remove irrelevant tags as we only need text
            for irrelevant in soup.body.find_all(["script", "style", "img", "input"]):
                irrelevant.decompose()
            
            self.text = soup.body.get_text(separator='\n', strip=True) if soup.body else ""

            # Extract links
            links = [link.get('href') for link in soup.find_all('a')]
            self.links = [link for link in links if link]

        except Exception as e:
            print(f"An error occurred while scraping: {e}")
    
    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Content:\n{self.text}\n\n"


In [ ]:
web = Website("https://edwarddonner.com")
web.links

In [ ]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [ ]:
print(link_system_prompt)

In [ ]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links. Don't include json term in output\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [ ]:
def get_prompt(website):
    return f"context: {link_system_prompt}, prompt: {get_links_user_prompt(website)}"

In [ ]:
def get_links(url):
    website = Website(url)
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(get_prompt(website))
    result = json.loads(response.text)
    return result

In [ ]:
get_links("https://anthropic.com")

## Step 2: Generate Brochure

In [ ]:
def get_details(url):
    result = "Landing Page:\n"
    website = Website(url)
    result += website.get_contents()
    links = get_links(url)
    for link in links['links']:
        result += f"\n\n{link['type']}:\n"
        result += Website(link['url']).get_contents()
    return result

In [ ]:
print(get_details("https://anthropic.com"))

In [ ]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [76]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [77]:
get_brochure_user_prompt("Anthropic", "https://anthropic.com")

'You are looking at a company called: Anthropic\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding Page:\nWebpage Title:\nHome \\ Anthropic\nWebpage Content:\nClaude\nOverview\nTeam\nEnterprise\nAPI\nPricing\nResearch\nCompany\nCareers\nNews\nTry Claude\nAI\nresearch\nand\nproducts\nthat put safety at the frontier\nClaude.ai\nMeet Claude 3.5 Sonnet\nClaude 3.5 Sonnet, our most intelligent AI model, is now available.\nTalk to Claude\nAPI\nBuild with Claude\nCreate AI-powered applications and custom experiences using Claude.\nLearn more\nAnnouncements\nIntroducing computer use, a new Claude 3.5 Sonnet, and Claude 3.5 Haiku\nOct 22, 2024\nModel updates\n3.5 Sonnet\n3.5 Haiku\nOur Work\nProduct\nClaude for Enterprise\nSep 4, 2024\nAlignment\n·\nResearch\nConstitutional AI: Harmlessness from AI Feedback\nDec 15, 2022\nAnnouncements\nCore Views on AI Safety: When, Why, What, and How\nMar 8, 

In [82]:
# get brochure from the prompt data

def generate_brochure(company_name, url):
    model = genai.GenerativeModel("gemini-1.5-flash", 
    system_instruction=system_prompt
    )
    response = model.generate_content(
        get_brochure_user_prompt(company_name, url), 
    )
    display(Markdown(response.text))

In [83]:
generate_brochure("Anthropic", "https://anthropic.com")

# Anthropic:  Building a Safer Future with AI

**A Brochure for Customers, Investors, and Recruits**

**(Image:  A compelling image showcasing Anthropic's team or technology)**

**Who We Are:**

Anthropic is a leading AI safety and research company based in San Francisco.  We're driven by a mission to ensure transformative AI helps people and society flourish. We achieve this by building reliable, interpretable, and steerable AI systems, conducting cutting-edge research, and collaborating with partners across various sectors.  Our interdisciplinary team comprises experts in machine learning, physics, policy, and product development, working together to address the opportunities and risks presented by advanced AI.


**What We Do:**

* **Develop cutting-edge AI models:** Anthropic is renowned for its Claude family of AI models, including the powerful Claude 3.5 Sonnet, designed for safety and reliability.  These models are available through our robust API, empowering businesses to integrate advanced AI capabilities into their applications and custom experiences.
* **Conduct groundbreaking research:** We treat AI safety as a systematic science, conducting and publishing research on interpretability, reinforcement learning from human feedback, and the broader societal impact of AI. This research informs our product development and helps shape industry best practices.
* **Foster collaboration:** We believe that achieving a safe and beneficial AI future requires collaboration. We actively engage with civil society, government, academia, nonprofits, and industry partners to share our findings and promote responsible AI development across the board.


**Our Culture:**

Anthropic is a collaborative, high-trust environment where diverse perspectives are valued. We prioritize open communication, intellectual honesty, and emotional maturity. We believe that our team's diverse experiences – ranging from NASA to startups to the armed forces – contribute to our innovative and supportive workplace. We value our mission and work together as "One big team" to achieve our shared goals.


**For Customers:**

Our Claude API offers access to state-of-the-art AI models designed for reliability and safety.  We provide enterprise-grade solutions to help businesses harness the power of AI responsibly and effectively. Learn more at [link to Anthropic's Enterprise page].


**For Investors:**

Anthropic is at the forefront of a rapidly evolving field, addressing critical challenges in AI safety and development. We are committed to delivering innovative, reliable AI systems and generating substantial returns through our groundbreaking research and product offerings.  [Optional: Add information about funding rounds or investment opportunities].


**For Recruits:**

Join a team of passionate individuals dedicated to building a responsible AI future. We offer challenging and rewarding careers for researchers, engineers, policy experts, and operational leaders.  Explore our open roles at [link to Anthropic's Careers page].  We're looking for individuals who are mission-driven, collaborative, and committed to building a better future.



**Contact Us:**

[Link to Anthropic's contact page]

[Anthropic logos and social media links]


In [84]:
# generate stream of content

def generate_brochure_stream(company_name, url):
    model = genai.GenerativeModel("gemini-1.5-flash",
    system_instruction=system_prompt
    )
    response = model.generate_content(
        get_brochure_user_prompt(company_name, url), 
        stream = True, 
    )
    result = ""
    display_handler = display(Markdown(""), display_id=True)
    for chunk in response:
        chunk.text.replace("'''", "").replace("markdown", "")
        result += chunk.text
        update_display(Markdown(result), display_id=display_handler.display_id)


In [85]:
generate_brochure_stream("Anthropic", "https://anthropic.com")

# Anthropic:  AI for Good

**A Brochure for Customers, Investors, and Recruits**

**(Image:  A compelling image showcasing Anthropic's team, technology, or a relevant graphic)**


**About Anthropic:**

Anthropic is a leading AI safety and research company based in San Francisco.  We're dedicated to building reliable, interpretable, and steerable AI systems that benefit humanity.  Our interdisciplinary team combines expertise in machine learning, physics, policy, and product development to create cutting-edge AI solutions and conduct groundbreaking research. We believe AI has the potential to fundamentally change the world, and we're committed to ensuring this transformation is safe and beneficial for all.

**Our Mission:** To ensure transformative AI helps people and society flourish.


**What We Do:**

* **Frontier AI Research:** We conduct leading-edge research in AI safety, exploring areas such as interpretability, reinforcement learning from human feedback, and the societal impact of AI.
* **Product Development:** We translate our research into practical, beneficial tools.  Our flagship product, Claude, is a powerful and reliable AI assistant available via API, empowering businesses, non-profits, and individuals.
* **Safety-Focused Approach:**  We treat AI safety as a systematic science, integrating research, product development, and knowledge sharing to continuously improve our systems and mitigate risks.
* **Collaboration and Partnerships:** We actively collaborate with civil society, governments, academia, and industry partners to promote widespread AI safety.


**For Customers:**

* **Claude:** Access powerful, reliable AI through our easy-to-use API.  Build AI-powered applications and custom experiences tailored to your needs. (Link to API documentation)
* **Enterprise Solutions:** Tailor-made AI solutions for businesses seeking reliable and safe AI integration. (Link to Enterprise page)


**For Investors:**

* **Market Leadership:** We're at the forefront of AI safety research and development, poised to capitalize on the growing demand for trustworthy AI solutions.
* **Strong Team:**  Our diverse and experienced team brings together leading experts from various fields, creating a powerful synergy for innovation.
* **Significant Impact:** Invest in a company that is making a tangible difference in shaping the future of AI for the better.


**For Recruits:**

* **Join a Mission-Driven Team:** Be part of a company dedicated to building a safer and more beneficial future with AI.
* **Collaborative Environment:** Work alongside leading researchers, engineers, and policy experts in a supportive and collaborative environment.  We value intellectual openness and emotional maturity.
* **Diverse Opportunities:**  Explore a variety of roles across research, engineering, policy, operations, and more.
* **Company Culture:** We foster a high-trust environment where individual contributions are valued and celebrated.  We believe collaboration is key and love to nurture a thriving work environment (we even love plants!).


**(Contact Information: Website, Email, Social Media Links)**

**(Footer: © 2025 Anthropic PBC)**
